# ReACT Agent

In this notebook we will look into experimenting with ReACT Agent with Simple Calculator Tools and RAG QueryEngine Tools.

In [ ]:
!pip install llama-index
!pip install llama-index-llms-anthropic
!pip install llama-index-llms-mistralai
!pip install llama-index-llms-openai-like #for local llm server

## Simple Calculator Tools

In [3]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.mistralai import MistralAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.llms.openai_like import OpenAILike
import os
import dotenv

## Load env

In [2]:
dotenv.load_dotenv()

True

### Define Tools

In [4]:
import datetime


def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b

def get_current_datetime(*args, **kwargs) -> str :
    """Get current datetime"""
    return datetime.datetime.now().strftime('%m/%d/%Y, %H:%M:%S')

In [6]:
multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)
get_current_datetime_tool = FunctionTool.from_defaults(fn=get_current_datetime, description="Get the current date time, it takes no input")

####  Local LLM

In [7]:
from typing import List
# llm =  OpenAILike(
#         model='llama-3.2-1b-instruct',
#         api_base='http://127.0.0.1:1234/v1',
#         api_key="fake",
#     )

llm = OpenAI(
    model='gpt-4o',
    api_key=os.environ.get('OPENAI_KEY'),
)


agent = ReActAgent.from_tools([multiply_tool, add_tool, subtract_tool, get_current_datetime_tool], llm=llm, verbose=True, context='For anything that you cannot answer with existing tools, simply reply "Sorry, I cannot comply to your enquiry"')

In [8]:

from llama_index.core.agent import AgentChatResponse
from llama_index.core.llms import MessageRole, ChatMessage

chat_histories:List[ChatMessage] = []

def chat(chat: str):
    response: AgentChatResponse = agent.chat(chat, chat_history=chat_histories)
    
    chat_histories.append(
        ChatMessage(
          content=chat,
          role=MessageRole.USER
        )
        
    )
    
    chat_histories.append(
        ChatMessage(
            content=response.response,
            role=MessageRole.ASSISTANT,
        )  
    )
    
    return response

Starts a conversation

In [9]:
response = chat('whats the time now')

> Running step 8dea7551-a7fb-47b8-b7f7-67daaaee04af. Step input: whats the time now
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: get_current_datetime
Action Input: {}
Observation: 02/18/2025, 20:38:04
> Running step d82fc355-1bb6-4fde-b42b-f5deda88f395. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The current time is 20:38:04 on February 18, 2025.


Trying questions that are out of scope

In [10]:
response = chat('whats the weather today?')

> Running step 635ba052-24f1-428c-af6b-5b8bb54f5d9f. Step input: whats the weather today?
Thought: The current language of the user is English. I cannot determine the weather with the provided tools.
Answer: Sorry, I cannot comply to your enquiry.


#### Anthropic Sonnet

In [ ]:
os.environ['ANTHROPIC_API_KEY'] = 'YOUR ANTHROPIC API KEY'

In [ ]:
llm = Anthropic(model="claude-3-sonnet-20240229")
agent = ReActAgent.from_tools([multiply_tool, add_tool, subtract_tool], llm=llm, verbose=True)

In [ ]:
response = agent.chat("What is 20+(2*4)? Calculate step by step.")

Thought: The current language of the user is: English. I need to use tools to help me answer the question step by step.

Action: multiply
Action Input: {"a": 2, "b": 4}

Observation: 8

Thought: I now need to add 20 to the result of 2*4, which is 8.
Action: add
Action Input: {"a": 20, "b": 8}

Observation: 28

Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: To calculate 20+(2*4) step by step:
1) First, calculate 2*4 using the multiply tool: 2 * 4 = 8
2) Then, add 20 to the result using the add tool: 20 + 8 = 28
Therefore, the answer is 28.


In [ ]:
print(response)

To calculate 20+(2*4) step by step:
1) First, calculate 2*4 using the multiply tool: 2 * 4 = 8
2) Then, add 20 to the result using the add tool: 20 + 8 = 28
Therefore, the answer is 28.


In [ ]:
response = agent.chat("40 + (100 - 30)*5 ? Calculate step by step.")

Thought: The current language of the user is: English. I need to use tools to calculate this step-by-step.

Action: subtract
Action Input: {"a": 100, "b": 30}

Observation: 70

Thought: Now I need to multiply the result by 5.

Action: multiply 
Action Input: {"a": 70, "b": 5}

Observation: 350

Thought: Finally, I need to add 40 to the result.

Action: add
Action Input: {"a": 350, "b": 40}  

Observation: 390

Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: To calculate 40 + (100 - 30)*5 step-by-step:
1) Calculate 100 - 30 = 70 
2) Multiply 70 by 5 to get 350
3) Add 40 to 350 to get 390
Therefore, the final answer is 390.


In [ ]:
print(response)

To calculate 40 + (100 - 30)*5 step-by-step:
1) Calculate 100 - 30 = 70 
2) Multiply 70 by 5 to get 350
3) Add 40 to 350 to get 390
Therefore, the final answer is 390.


#### MistralAI

In [ ]:
os.environ['MISTRAL_API_KEY'] = 'YOUR MISTRAL API KEY'

In [ ]:
llm = MistralAI(model="mistral-large-latest")
agent = ReActAgent.from_tools([multiply_tool, add_tool, subtract_tool], llm=llm, verbose=True)

In [ ]:
response = agent.chat("What is 20+(2*4)? Calculate step by step.")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question. The question requires me to perform multiplication first due to the order of operations (BIDMAS/BODMAS), then addition.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
Thought: The current language of the user is: English. I have the result of the multiplication. Now, I need to add the result to 20.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The result of 20 + (2 * 4) is 28.


In [ ]:
print(response)

The result of 20 + (2 * 4) is 28.


In [ ]:
response = agent.chat("40 + (100 - 30)*5 ? Calculate step by step.")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: subtract
Action Input: {'a': 100, 'b': 30}
Observation: 70
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 70, 'b': 5}
Observation: 350
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 40, 'b': 350}
Observation: 390
Thought: The current language of the user is: English. I have enough information to answer the question without using any more tools.
Answer: The result of 40 + (100 - 30) \* 5 is 390.


In [ ]:
print(response)

The result of 40 + (100 - 30) \* 5 is 390.


### Check the Prompt

In [ ]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do t

## RAG QueryEngine Tools

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [ ]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

### Download Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-04-12 20:27:33--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.08s   

2024-04-12 20:27:33 (22.7 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-04-12 20:27:34--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connec

### Load Data

In [ ]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

### Create Index

We will by default use OpenAI Embeddings for building index.

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

### Define Tools

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

#### OpenAI GPT-4

In [ ]:
llm = OpenAI(model="gpt-4")
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

In [ ]:
response = agent.chat(
    "Compare the revenue growth of Uber and Lyft in 2021."
)

Thought: To compare the revenue growth of Uber and Lyft in 2021, I need to use the 'uber_10k' and 'lyft_10k' tools to get the financial information for both companies. I'll start with Uber.
Action: uber_10k
Action Input: {'input': "What was Uber's revenue growth in 2021?"}
Observation: Uber's revenue grew by 57% in 2021.
Thought: Now that I have the revenue growth for Uber, I need to get the same information for Lyft. I'll use the 'lyft_10k' tool for this.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Lyft's revenue increased by 36% in 2021 compared to the prior year.
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: In 2021, Uber's revenue grew by 57%, while Lyft's revenue increased by 36%. Therefore, Uber had a higher revenue growth compared to Lyft in 2021.


In [ ]:
print(str(response))

In 2021, Uber's revenue grew by 57%, while Lyft's revenue increased by 36%. Therefore, Uber had a higher revenue growth compared to Lyft in 2021.


#### Anthropic Sonnet

In [ ]:
llm = Anthropic(model="claude-3-sonnet-20240229")
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

In [ ]:
response = agent.chat(
    "Compare the revenue growth of Uber and Lyft in 2021."
)

Thought: The current language of the user is: English. I need to use tools to get information about Uber and Lyft's revenue growth in 2021 to answer this question.
Action: uber_10k
Action Input: {'input': "What was Uber's revenue growth in 2021?"}
Observation: Uber's revenue grew by 57% in 2021.
Thought: I now have information about Uber's revenue growth in 2021, but still need Lyft's revenue growth to compare the two companies.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Lyft's revenue increased by 36% in 2021 compared to the prior year.
Thought: I now have the revenue growth information for both Uber and Lyft in 2021. I can answer the question without using any more tools.
Answer: In 2021, Uber's revenue grew at a faster rate of 57% compared to Lyft's revenue growth of 36%.


In [ ]:
print(response)

In 2021, Uber's revenue grew at a faster rate of 57% compared to Lyft's revenue growth of 36%.
